In [75]:
from carbon.sdk import CarbonSDK, Tokens as T
from carbon import CarbonOrderUI, CarbonSimulatorUI, CarbonPair as P
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# API Testing [NB02]

_this is currently the main reference notebook, showcasing all API endpoints_

In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [76]:
SDK = CarbonSDK(disclaimer=True, verbose=True, Tokens=T)
SDK.version().get("msg")



    This is a demo and testing wrapper for the Carbon SDK. YOU MUST 
    NOT USE THIS SDK, OR ITS ASSOCIATED NODEJS CODE IN PRODUCTION 
    WHEN FUNDS ARE AT RISK. See the disclaimer on the Carbon SDK 
    NodeJS server for more information.
    
[req0] method=get, url=http://localhost:3118/, params={}


'Carbon SDK v0.9 (7/Mar/2022)'

## Generic API calls

### Call using `req`
the only difference between `req` and `req0` is that the leading `api` must be omitted)

In [77]:
help(SDK.req)

Help on method req in module carbon.sdk.sdkbase:

req(ep, params=None, method=None) method of carbon.sdk.carbonsdk.CarbonSDK instance
    execute a GET or POST request to the actual API
    
    :ep:        API endpoint, including the sync/async prefix (eg 'scall/plus')
    :params:    params transmitted to the API (POST only)
    :method:    self.GET or self.POST



In [78]:
r = SDK.req("scall/qplus", params={"a":3, "b":5}, method=SDK.POST)
r.json()

[req0] method=post, url=http://localhost:3118/api/scall/qplus, params={'a': 3, 'b': 5}


{'success': True, 'data': {'func': 'qplus', 'a': 3, 'b': 5, 'result': 8}}

### Call using `call`
call only works for the `scall` / `ascall` subapis; which one is chosen, and how results are returned, depends on the `sync` parameter; also whether to GET or POST can be implied from the existence of the `params` parameter

In [79]:
help(SDK.call)

Help on method call in module carbon.sdk.sdkbase:

call(ep, params=None, method=None, sync=None) method of carbon.sdk.carbonsdk.CarbonSDK instance
    execute a GET or POST request to the (sync or async) call API 
    
    :ep:        API endpoint, with the sync/async prefix (eg 'plus')
    :params:    params transmitted to the API (POST only)
    :method:    self.GET or self.POST
    :sync:      whether to call sync or async



In [80]:
r = SDK.call("qplus", params={"a":3, "b":5}, sync=True)
r.json()

[req0] method=post, url=http://localhost:3118/api/scall/qplus, params={'a': 3, 'b': 5}


{'success': True, 'data': {'func': 'qplus', 'a': 3, 'b': 5, 'result': 8}}

## Market information endpoints

#### `pairs` -- information about all available pairs

The `pairs` endpoint provides a list of all available pairs, or rather their contract addresses. If a `Tokens` container is available on the object the addresses are converted into token objects (where available).

In [81]:
pairs, pairs_s = SDK.pairs(inclstr=True)
print(pairs_s)
pairs[:1]

[req0] method=post, url=http://localhost:3118/api/scall/pairs, params={}
USDC/ETH


[[SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'),
  SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH')]]

#### `hasLiquidityByPair` -- whether a given pair has any liquidity

In [82]:
SDK.hasLiquidityByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a)

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


True

In [83]:
SDK.hasLiquidityByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a)

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


True

#### `mHasLiquidityByPair` -- whether a given pair has any liquidity (modified)

In [84]:
SDK.mHasLiquidityByPair(pair=P("ETH/USDC"), AMMsells="ETH")

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


True

In [85]:
SDK.mHasLiquidityByPair(pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


True

#### mHasLiquidityByPairs

In [86]:
SDK.mHasLiquidityByPairs(pairs=[P("ETH/USDC"), P("BNT/USDT"), P("LINK/AAVE")])

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'pairList': [{'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}, {'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}, {'sourceToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c', 'targetToken': '0xdac17f958d2ee523a2206206994597c13d831ec7'}, {'sourceToken': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'targetToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c'}, {'sourceToken': '0x514910771af9ca656af840dff83e8264ecf986ca', 'targetToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9'}, {'sourceToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'targetToken': '0x514910771af9ca656af840dff83e8264ecf986ca'}]}


({'pair': P('ETH/USDC'), 'AMMsells': 'USDC', 'hasLiquidity': True},
 {'pair': P('ETH/USDC'), 'AMMsells': 'ETH', 'hasLiquidity': True},
 {'pair': P('BNT/USDT'), 'AMMsells': 'USDT', 'hasLiquidity': False},
 {'pair': P('BNT/USDT'), 'AMMsells': 'BNT', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'AAVE', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'LINK', 'hasLiquidity': False})

In [87]:
SDK.mHasLiquidityByPairs(pairs=["ETH/USDC", "BNT/USDT", "LINK/AAVE"])

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'pairList': [{'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}, {'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}, {'sourceToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c', 'targetToken': '0xdac17f958d2ee523a2206206994597c13d831ec7'}, {'sourceToken': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'targetToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c'}, {'sourceToken': '0x514910771af9ca656af840dff83e8264ecf986ca', 'targetToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9'}, {'sourceToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'targetToken': '0x514910771af9ca656af840dff83e8264ecf986ca'}]}


({'pair': P('ETH/USDC'), 'AMMsells': 'USDC', 'hasLiquidity': True},
 {'pair': P('ETH/USDC'), 'AMMsells': 'ETH', 'hasLiquidity': True},
 {'pair': P('BNT/USDT'), 'AMMsells': 'USDT', 'hasLiquidity': False},
 {'pair': P('BNT/USDT'), 'AMMsells': 'BNT', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'AAVE', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'LINK', 'hasLiquidity': False})

In [88]:
SDK.mHasLiquidityByPairs(pairs="ETH/USDC, BNT/USDC, LINK/AAVE")

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'pairList': [{'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}, {'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}, {'sourceToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}, {'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c'}, {'sourceToken': '0x514910771af9ca656af840dff83e8264ecf986ca', 'targetToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9'}, {'sourceToken': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'targetToken': '0x514910771af9ca656af840dff83e8264ecf986ca'}]}


({'pair': P('ETH/USDC'), 'AMMsells': 'USDC', 'hasLiquidity': True},
 {'pair': P('ETH/USDC'), 'AMMsells': 'ETH', 'hasLiquidity': True},
 {'pair': P('BNT/USDC'), 'AMMsells': 'USDC', 'hasLiquidity': False},
 {'pair': P('BNT/USDC'), 'AMMsells': 'BNT', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'AAVE', 'hasLiquidity': False},
 {'pair': P('LINK/AAVE'), 'AMMsells': 'LINK', 'hasLiquidity': False})

In [89]:
SDK.mHasLiquidityByPairs(pairs=pairs_s)

[req0] method=post, url=http://localhost:3118/api/scall/has_liquidity_by_pair, params={'pairList': [{'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}, {'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}]}


({'pair': P('USDC/ETH'), 'AMMsells': 'ETH', 'hasLiquidity': True},
 {'pair': P('USDC/ETH'), 'AMMsells': 'USDC', 'hasLiquidity': True})

#### `getLiquidityByPair` -- the amount of liquidity in a pair

in units of the target token

In [90]:
SDK.getLiquidityByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


101.0

In [91]:
SDK.getLiquidityByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


1500.0

#### `mGetLiquidityByPair` -- the amount of liquidity in a pair (modified)

this function -- like all other `mGet` functions -- is modified to be better in line with the conventions of the Carbon Simulator; it returns a `PairLiquidity` object that is also used for `mGetRateLiquidityDepthByPair`'; to indicate that the liquidity here is the total liquidity the `rate` field is set to zero.

In [92]:
SDK.mGetLiquidityByPair(pair=P("ETH/USDC"), AMMsells="ETH")

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


PairLiquidity(amount=101.0, unit='ETH', AMMsells='ETH', bidAsk='ask', pair=P('ETH/USDC'), rate=None, inverted=None, islist=None)

In [93]:
SDK.mGetLiquidityByPair(pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


PairLiquidity(amount=1500.0, unit='USDC', AMMsells='USDC', bidAsk='bid', pair=P('ETH/USDC'), rate=None, inverted=None, islist=None)

#### `getMinRateByPair` and `getMaxRateByPair` -- get the min and max exchange rates

gets the minimal and maximal rate availble in posisitions in this pair in this direction (used to determine the boundaries for the subsequenct calls to `getRateLiquidityDepthByPair`; `sourceToken` is base token and `targetToken` is quote token, ie in other words the prices are expressed in `targetToken` units


In [94]:
SDK.getMinRateByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


1499.999999795006

In [95]:
SDK.getMaxRateByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


1599.9999998788553

In [96]:
SDK.getMinRateByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


0.0003333333333333297

In [97]:
SDK.getMaxRateByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a)

[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


0.0004

#### `mGetRangeByPair` -- combining min and max rate

The `mGetRangeByPair` combines min and max rate and uses the correct quote direction of the `CarbonPair`. It requires a `TokenContainer` object that allows to associate the ticker with the token address.

In [98]:
SDK.mGetRangeByPair(pair=P("ETH/USDC"), AMMsells="ETH")

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}


RangeByPair(startRate=2500.0, endRate=3000.0000000000327, AMMsells='ETH', AMMbuys='USDC', bidAsk='ask', pair=P('ETH/USDC'), inverted=False)

In [99]:
SDK.mGetRangeByPair(pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}


RangeByPair(startRate=1599.9999998788553, endRate=1499.999999795006, AMMsells='USDC', AMMbuys='ETH', bidAsk='bid', pair=P('ETH/USDC'), inverted=True)

#### `getRateLiquidityDepthByPair` -- get the liquidity at a certain exchange rate

used to determine the liquidity depth chart; use `getMinRateByPair` and `getMaxRateByPair` to determine range; rate is given in target token per source token, and the amount in target token; the AMM buys (and trader sells) the source token

In [100]:
(SDK.getLiquidityByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a),
 SDK.getRateLiquidityDepthByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a, rate=800),
 SDK.getRateLiquidityDepthByPair(sourceTokenAddr=T.ETH.a, targetTokenAddr=T.USDC.a, rate=2000))

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'rate': '800'}
[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'rate': '2000'}


(1500.0, 1500.0, 0.0)

In [101]:
(
 SDK.getLiquidityByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a),    
 SDK.getRateLiquidityDepthByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a, rate=1/3000),
 SDK.getRateLiquidityDepthByPair(sourceTokenAddr=T.USDC.a, targetTokenAddr=T.ETH.a, rate=1/2550))

[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'rate': '0.0003333333333333333'}
[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'rate': '0.000392156862745098'}


(101.0, 1.0, 0.5073532357002739)

#### `mGetRateLiquidityDepthByPair` -- get the liquidity at a certain exchange rate (modified)

returns a `PairLiquidity` object like `mGetLiquidityByPair`

In [102]:
SDK.mGetRateLiquidityDepthByPair(rate=3000, pair=P("ETH/USDC"), AMMsells="ETH")

[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'rate': '0.0003333333333333333'}


PairLiquidity(amount=1.0, unit='ETH', AMMsells='ETH', bidAsk='ask', pair=P('ETH/USDC'), rate=3000, inverted=True, islist=False)

In [103]:
SDK.mGetRateLiquidityDepthByPair(rate=2550, pair="ETH/USDC", AMMsells="ETH")

[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'rate': '0.000392156862745098'}


PairLiquidity(amount=0.5073532357002739, unit='ETH', AMMsells='ETH', bidAsk='ask', pair=P('ETH/USDC'), rate=2550, inverted=True, islist=False)

In [104]:
SDK.mGetRateLiquidityDepthByPair(rate=2000, pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'rate': '2000'}


PairLiquidity(amount=0.0, unit='USDC', AMMsells='USDC', bidAsk='bid', pair=P('ETH/USDC'), rate=2000, inverted=False, islist=False)

In [105]:
SDK.mGetRateLiquidityDepthByPair(rate=800, pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'rate': '800'}


PairLiquidity(amount=1500.0, unit='USDC', AMMsells='USDC', bidAsk='bid', pair=P('ETH/USDC'), rate=800, inverted=False, islist=False)

In [106]:
SDK.mGetRateLiquidityDepthByPair(rate=[800, 1000, 1100], pair=P("ETH/USDC"), AMMsells="USDC")

[req0] method=post, url=http://localhost:3118/api/scall/get_rate_liquidity_depth_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'rate': ['800', '1000', '1100']}


PairLiquidity(amount=[1500.0, 1500.0, 1500.0], unit='USDC', AMMsells='USDC', bidAsk='bid', pair=P('ETH/USDC'), rate=[800, 1000, 1100], inverted=False, islist=True)

## Maker-user information endpoints

_Maker users_ are user that provide liquidity to the market, ie who create strategies. Those endpoint look at their portfolio information.

#### `addr` -- get the address of the wallet in the SDK server (effectively user address)

In [107]:
SDK.req("addr").json()

[req0] method=get, url=http://localhost:3118/api/addr, params={}


{'success': True, 'data': '0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee'}

In [108]:
sdkaddr = SDK.addr()
sdkaddr,

[req0] method=get, url=http://localhost:3118/api/addr, params={}


('0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee',)

#### `getUserStrategies` -- get all strategies of the user

returns all strategies for a given user (default: the waller related to the SDK server); use `reformatStrategy` to convert the returned value into types more reasonable for Python analysis (notably, `BigNumber` dicts to `int`)

In [109]:
data = SDK.getUserStrategies(user=sdkaddr)
print("Number of strategies:", len(data))
print("Strategy ids", [SDK.bn2int(d["id"]) for d in data])
data[0]

[req0] method=post, url=http://localhost:3118/api/scall/get_user_strategies, params={'user': '0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee'}
Number of strategies: 2
Strategy ids [1, 2]


{'id': {'type': 'BigNumber', 'hex': '0x01'},
 'baseToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
 'quoteToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 'buyPriceLow': '1499.999999795006131230270249480266202369446401111707789510774091468192636966705322265625',
 'buyPriceHigh': '1499.999999795006131230270249480266202369446401111707789510774091468192636966705322265625',
 'buyBudget': '1000',
 'sellPriceLow': '3000.000000000032668092946766964885086163396166485268987541372488603501882527572149029185205026847252',
 'sellPriceHigh': '3000.000000000032668092946766964885086163396166485268987541372488603501882527572149029185205026847252',
 'sellBudget': '100',
 'encoded': {'id': {'type': 'BigNumber', 'hex': '0x01'},
  'token0': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
  'token1': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'order0': {'y': {'type': 'BigNumber', 'hex': '0x056bc75e2d63100000'},
   'z': {'type': 'BigNumber', 'hex': '0x056bc75e2d63100000'},
   'A': {'type': 'B

The strategies can be imported into the Carbon Simulator into the `CarbonOrderUI` class and into the `CarbonSimulatorUI` class

In [110]:
obuy, osell = CarbonOrderUI.from_SDK(data[0])
obuy, osell

(CarbonOrderUI(pair=P('TKNB/TKNQ'), tkn=TKNQ, B=38.72983345942771, S=0.0, yint=1000.0, y=1000.0, id=1-b, linked=<1-s>),
 CarbonOrderUI(pair=P('TKNB/TKNQ'), tkn=TKNB, B=0.018257418583505436, S=0.0, yint=100.0, y=100.0, id=1-s, linked=<1-b>))

In [111]:
Sim = CarbonSimulatorUI()
o = Sim.add_fromsdk(data[0], 6)["orders"]
o

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,TKNBTKNQ,TKNB,100.0,100.0,TKNB,False,3000.0,3000.0,3000.0,TKNQ per TKNB,1,NaN
1,1,TKNBTKNQ,TKNQ,1000.0,1000.0,TKNQ,False,1500.0,1500.0,1500.0,TKNQ per TKNB,0,NaN


In [112]:
data

[{'id': {'type': 'BigNumber', 'hex': '0x01'},
  'baseToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
  'quoteToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'buyPriceLow': '1499.999999795006131230270249480266202369446401111707789510774091468192636966705322265625',
  'buyPriceHigh': '1499.999999795006131230270249480266202369446401111707789510774091468192636966705322265625',
  'buyBudget': '1000',
  'sellPriceLow': '3000.000000000032668092946766964885086163396166485268987541372488603501882527572149029185205026847252',
  'sellPriceHigh': '3000.000000000032668092946766964885086163396166485268987541372488603501882527572149029185205026847252',
  'sellBudget': '100',
  'encoded': {'id': {'type': 'BigNumber', 'hex': '0x01'},
   'token0': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'token1': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'order0': {'y': {'type': 'BigNumber', 'hex': '0x056bc75e2d63100000'},
    'z': {'type': 'BigNumber', 'hex': '0x056bc75e2d63100000'},
    '

#### `mGetUserStrategies` -- get all strategies of the user (modified)

applies `reformateStrategy` to all strategies

In [113]:
data = SDK.mGetUserStrategies(user=sdkaddr)
data

[req0] method=post, url=http://localhost:3118/api/scall/get_user_strategies, params={'user': '0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee'}


(Strategy(sid=1, baseToken='ETH', quoteToken='USDC', buyPriceLow=1500.0, buyPriceHigh=1500.0, buyBudget=1000.0, sellPriceLow=3000.0, sellPriceHigh=3000.0, sellBudget=100.0, encoded=EncodedStrategy(sid=1, order0=EncodedOrder(token='ETH', y=100000000000000000000, z=100000000000000000000, A=0, B=4378954682110917), order1=EncodedOrder(token='USDC', y=1000000000, z=1000000000, A=0, B=10901478971))),
 Strategy(sid=2, baseToken='ETH', quoteToken='USDC', buyPriceLow=1500.0, buyPriceHigh=1600.0, buyBudget=500.0, sellPriceLow=2500.0, sellPriceHigh=2600.0, sellBudget=1.0, encoded=EncodedStrategy(sid=2, order0=EncodedOrder(token='ETH', y=1000000000000000000, z=1000000000000000000, A=2746792519209792, B=4390587127987123), order1=EncodedOrder(token='USDC', y=500000000, z=500000000, A=357520097, B=10901478971))))

In [114]:
SDK.reformatStrategy(data[0])["encoded"]

{'id': 1,
 'token0': 'eth',
 'token1': 'usdc',
 'order0': {'y': 100000000000000000000,
  'z': 100000000000000000000,
  'A': 0,
  'B': 4378954682110917},
 'order1': {'y': 1000000000, 'z': 1000000000, 'A': 0, 'B': 10901478971}}

## Maker-user action endpoints

Those endpoints allow maker users to create, modify and delete their strategies. Note that all active endpoints return unsigned transactions. Those are then submitted to the endpoint `signsubmittx` to be signed and submitted. If `sign` is `True`, the transaction is signed using the server wallet private key (with the obvious security implications). Alternatively an already signed transaction can be submitted. 

#### `createBuySellStrategy` -- creates a strategy



In [115]:
data = SDK.createBuySellStrategy(baseToken=T.ETH.a, quoteToken=T.USDC.a, 
                              buyPriceLow =1500, buyPriceHigh =1600, buyBudget =500,    # buy 500 USDC worth of ETH from 1500..1600
                              sellPriceLow=2500, sellPriceHigh=2600, sellBudget=   1,   # sell 1 ETH from 2500..2600
                              overrides=None)

[req0] method=post, url=http://localhost:3118/api/scall/create_buy_sell_strategy, params={'baseToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'quoteToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'buyPriceLow': '1500', 'buyPriceHigh': '1600', 'buyBudget': '500', 'sellPriceLow': '2500', 'sellPriceHigh': '2600', 'sellBudget': '1', 'overrides': {'gasLimit': 999999999}}


In [116]:
SDK.signsubmittx(data, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf727473a000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb480000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000009c23178611340000000000000000000000000000000000000000000000000000f99373a1e7bb3000000000000000000000000000000000000000000000000000000001dcd6500000000000000000000000000000000000000000000000000000000001dcd650000000000000000000000000000000000000000000000000000000000154f52e10000000000000000000000000000000000000000000000000000000289c75e3b', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}, 'value': {'type': 'BigNumber', 'hex': '0x0de0b6b3a7640000'}}, True: True}


{'txReceipt': {'hash': '0xd462a2b28b795786060dcd8157bde8463924f016993d03d6e0f0b6d2f81af20f',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

#### check user strategy IDs

In [117]:
data = SDK.getUserStrategies(user=sdkaddr)
ids = [SDK.bn2int(d["id"]) for d in data]
print("Number of strategies:", len(data))
print("Strategy ids", ids)

[req0] method=post, url=http://localhost:3118/api/scall/get_user_strategies, params={'user': '0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee'}
Number of strategies: 2
Strategy ids [1, 2]


#### `updateStrategy` -- updates a strategy [yet to be connected]

In [118]:
r = SDK.updateStrategy(strategyId=None, encoded=None, baseToken=None, quoteToken=None, update=None, 
                       buyMarginalPrice=None, sellMarginalPrice=None, overrides=None, sync=True)
r.json()

[req0] method=post, url=http://localhost:3118/api/scall/update_strategy, params={'strategyId': 'None', 'encoded': None, 'baseToken': 'None', 'quoteToken': 'None', 'update': False, 'buyMarginalPrice': 'None', 'sellMarginalPrice': 'None', 'overrides': {'gasLimit': 999999999}}


{'success': True, 'data': {'result': 'not implemented yet [async]'}}

#### `deleteStrategy` -- deletes a strategy

Note: this deletes the strategy that has been created above. If this fails with an out-of-gas error then the reason may be that the strategy has not been created yet. Note that in this case the server may crash which is something we are working on. The last cell below shows, again, the user strategies.

In [119]:
data = SDK.deleteStrategy(strategyId=ids[-1])
data

[req0] method=post, url=http://localhost:3118/api/scall/delete_strategy, params={'strategyId': '2', 'overrides': {'gasLimit': 999999999}}


{'data': '0x873020370000000000000000000000000000000000000000000000000000000000000002',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6'}

In [120]:
#SDK.signsubmittx(data, sign=True)

In [121]:
data = SDK.getUserStrategies(user=sdkaddr)
ids = [SDK.bn2int(d["id"]) for d in data]
print("Number of strategies:", len(data))
print("Strategy ids", ids)

[req0] method=post, url=http://localhost:3118/api/scall/get_user_strategies, params={'user': '0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee'}
Number of strategies: 2
Strategy ids [1, 2]


**Note the weird error message if you try to delete a strategy that does not exist**

In [122]:
data = SDK.deleteStrategy(strategyId=1234567890)
try:
    rr = SDK.signsubmittx(data, sign=True)
except Exception as e:
    print(r)
    rr = None
rr

[req0] method=post, url=http://localhost:3118/api/scall/delete_strategy, params={'strategyId': '1234567890', 'overrides': {'gasLimit': 999999999}}
[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0x8730203700000000000000000000000000000000000000000000000000000000499602d2', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6'}, True: True}
<Response [200]>


## Taker-user (trader) action endpoints

Those endpoints allow taker users (ie traders) to trade against existing liquidity


### Integrated endpoint

#### `getTradeData` -- integrated function (here used "by source")

The endpoint `getTradeData` is the only endpoint usually needed to compose a trade. It internally calls the functions `tbd`. Those functions are also exposed as individual endpoints which allows to optimized the process and to obtain a more reactive code.


In [123]:
data = SDK.getTradeData(sourceToken=T.WBTC.a, targetToken=T.USDC.a, amount=0.001, tradeByTargetAmount=False, filter=None)
tradeActions = [SDK.bn2intd(d) for d in data["tradeActions"]]
print(tradeActions)
data

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'amount': '0.001', 'tradeByTargetAmount': False, 'filter': []}
[]


{'tradeActions': [],
 'actionsTokenRes': [],
 'totalSourceAmount': '0',
 'totalTargetAmount': '0',
 'effectiveRate': '0',
 'actionsWei': []}

#### `composeTradeBySourceTransaction` -- create transaction "by source"

Important: if this one fails there may be an issue with the approval. Need to approve the token first which can be done on the web app.

**TODO:HOW DO I SEND ETH WITH THE TRANSACTION?**

In [124]:
data = SDK.composeTradeBySourceTransaction(sourceToken=T.WBTC.a, targetToken=T.USDC.a, tradeActions=tradeActions, 
                                        deadline=None, minReturn=None, overrides=None)
data

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_source_transaction, params={'sourceToken': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [], 'deadline': '4294967295', 'minReturn': '1', 'overrides': {'gasLimit': 999999999}}


{'data': '0xf1c5e0140000000000000000000000002260fac5e5542a773aa44fbcfedf7c193bc2c599000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000000000f42400000000000000000000000000000000000000000000000000000000000000000',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}

In [125]:
data = SDK.signsubmittx(data, sign=True)
data 

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf1c5e0140000000000000000000000002260fac5e5542a773aa44fbcfedf7c193bc2c599000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000000000f42400000000000000000000000000000000000000000000000000000000000000000', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}, True: True}


{'txReceipt': {'hash': '0x962b68d71b38bf22a45919ffd27b0c7cf55012a090acd22a52ac021820dc265e',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [126]:
data = SDK.composeTradeByTargetTransaction(sourceToken=T.WBTC.a, targetToken=T.USDC.a, tradeActions=tradeActions, 
                                        deadline=None, maxInput=None, overrides=None)
data

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_target_transaction, params={'sourceToken': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [], 'deadline': '4294967295', 'maxInput': '340282366920938463463374607431768211455', 'overrides': {'gasLimit': 999999999}}


RuntimeError: ERROR: {'success': False, 'error': 'value out-of-bounds (argument="maxInput", value="34028236692093846346337460743176821145500000000", code=INVALID_ARGUMENT, version=abi/5.7.0)'}

#### `getTradeData` -- integrated function (here used "by target")

**TODO**

#### `composeTradeByTargetTransaction` -- create transaction "by target"

**TODO**

### Step-by-step endpoints

**THE BELOW DESCRIPTION IS NOT CORRECT, BUT YOU DO NOT GENERALLY NEED THE STEP BY STEP ANYWAY**

#### `getMatchParams` -- order matching step 1

This is the first step in the matching process executed by `getTradeData`. It takes the desired trade, and returns amongst other data the dict of `orders` against which to much. This dict has as key the `orderid` and as value the order parameters `AByz`. The `orders` part of the returned dict can be formatted into Python suitable values (notably, `BigNumber` to `int`) using `reformatOrders`. A single order (and any dict for that matter) can be converted using `bn2intd`.

In [ ]:
r = SDK.getMatchParams(sourceToken=T.ETH.a, targetToken=T.USDC.a, amount=1, tradeByTargetAmount=False, sync=True)
d = r.json()["data"]
d

In [ ]:
SDK.reformatOrders(d["orders"])

In [ ]:
SDK.bn2intd(tuple(d["orders"].values())[0])

#### `getMatchActions` -- order matching step 2

the parameter `orders` is a list of encoded strategies (yzAB), in the same format as the one returned by `getMatchParams`. `tradeByTargetAmount` determines whether by source or by target, and `amountWei` is the respective source or target amount in token wei. It returns an array of trade actions `[{id, input, output}, ...]`. This function does work on arbitrary order data which does not necessarily correspond to actual orders on-chain. This function returns the trade actions.

In [ ]:
r = SDK.getMatchActions(amountWei=10000, tradeByTargetAmount=None, orders=None, sync=True)
r.json()

#### `composeTradeByTargetTransaction` and `composeTradeBySourceTransaction` -- order matching step 3

Those functions take the trade actions returned from `getMatchActions` and it returns `????`

In [ ]:
data = SDK.composeTradeByTargetTransaction(sourceToken=T.ETH.a, targetToken=T.USDC.a, tradeActions={}, 
                                        deadline=None, maxInput=None, overrides=None)
data

In [ ]:
SDK.composeTradeBySourceTransaction(sourceToken=T.ETH.a, targetToken=T.USDC.a, tradeActions={}, 
                                        deadline=None, minReturn=None, overrides=None)

#### `getTradeDataFromActions` -- order matching step 4

???

In [ ]:
SDK.getTradeDataFromActions(sourceToken=T.ETH.a, targetToken=T.USDC.a, tradeByTargetAmount=False, actionsWei=[])